In [1]:
import pandas as pd


from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.model_selection import cross_val_score


In [2]:
df = pd.read_pickle("dfReduced.pkl")  

df

,X0,X1,X2,X3,NRS,Sex,Side,Diabetes,NP,Weakness,Severity
0,-5.347156,-8.126492,-5.075822,-1.694237,2,1,1,0,0,0,0
1,-10.672576,0.588534,-3.676703,-3.019831,6,0,0,0,0,0,2
2,-10.591772,0.245347,-4.360149,-2.968246,3,0,1,0,0,0,0
3,-2.952554,-8.432422,0.892403,4.113887,4,1,1,0,0,0,0
4,-26.209848,-14.981759,-1.778205,-0.250675,2,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
1032,-0.297217,-7.512667,-3.551025,-1.158835,3,1,1,0,0,0,0
1033,8.662963,2.225581,0.721682,-1.037305,5,1,0,0,1,1,2
1034,17.587751,-0.589560,-2.768376,-0.311974,4,0,0,0,0,0,0
1035,-8.476801,-3.329811,-2.885255,3.708300,3,1,1,0,0,0,0


In [3]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,0:10], df.iloc[:,10], test_size=0.265)

In [4]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [5]:
from deap import base
from deap import creator
from deap import tools

# !pip install deap
import multiprocessing
import random
import matplotlib.pyplot as plt
import numpy as np
import time
import math

In [6]:
def LRParametersFeatures(numberFeatures, icls):
    genome = list()
    # C
    genome.append(random.uniform(0.01, 100.0))
    # solver
    genome.append(random.randint(0, 3))
    # fit_intercept
    genome.append(random.randint(0, 1))
    # class_weight
    genome.append(random.randint(0, 1))


    return icls(genome)

In [7]:
def LREvaluation(X_train, X_test, y_train, y_test, df_reduced, individual):
    
    solver_arr = ['newton-cg', 'lbfgs', 'liblinear', 'saga']
    fit_intercept_arr=[True, False]
    class_weight_arr=['balanced', None]
    
    
    estimator = LogisticRegression(C=individual[0],
                                  solver=solver_arr[individual[1]],
                                  dual=False, 
                                  fit_intercept=fit_intercept_arr[individual[2]],
                                  class_weight=class_weight_arr[individual[3]],
                                  max_iter=10000)
    
    estimator.fit(X_train, y_train)
    result = estimator.predict(X_test)
    
    ScoreCV = cross_val_score(estimator, df_reduced.iloc[:,0:10], df_reduced.iloc[:,10], cv=5)
    
    return ScoreCV.sum() / 5

In [8]:
def LRmutation(individual):
    numberParamer= random.randint(0,len(individual)-1)
    if numberParamer == 0:
        individual[0] = random.uniform(0.01, 100.0)
    elif numberParamer == 1:
        individual[1] = random.randint(0, 3)
    elif numberParamer == 2:
        individual[2] = random.randint(0, 1)
    elif numberParamer == 3:
        individual[3] = random.randint(0, 1)

In [ ]:
    sizePopulation = 10
    probabilityMutation = 0.2
    probabilityCrossover = 0.8
    numberIteration = 10
    numberElitism = 5
    processes = 16
    numberOfAtributtes = 4

    creator.create("FitnessMax", base.Fitness, weights=(1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMax)
    toolbox = base.Toolbox()

    #zrównoleglenie
#     pool = multiprocessing.Pool(processes=processes)
#     toolbox.register("map", pool.map)

    toolbox.register('individual', LRParametersFeatures, numberOfAtributtes, creator.Individual)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    toolbox.register("evaluate", LREvaluation,X_train, X_test, y_train, y_test, df)

    #SELEKCJA
    toolbox.register("select", tools.selTournament, tournsize=3)

    #KRZYŻOWANIE
    toolbox.register("mate", tools.cxOnePoint)

    toolbox.register("mutate", LRmutation)
    
    pop = toolbox.population(n=sizePopulation)
    fitnesses = list(toolbox.map(toolbox.evaluate, pop))

In [ ]:
for ind, fit in zip(pop, fitnesses):
        ind.fitness = fit

In [ ]:

    start=time.time()
    g=0
    while g < numberIteration:
        g = g + 1
        print(g)
        offspring = toolbox.select(pop, len(pop))
        offspring = list(map(toolbox.clone, offspring))
        listElitism = [] 
        for x in range(0, numberElitism):
            listElitism.append(tools.selBest(pop, 1)[0])
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < probabilityCrossover:
                toolbox.mate(child1, child2)
                child1.fitness = 0.0
                child2.fitness = 0.0          
        for mutant in offspring:
            if random.random() < probabilityMutation:
                toolbox.mutate(mutant)
                mutant.fitness = 0.0
        invalid_ind = [ind for ind in offspring if ind.fitness == 0.0]
        fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness = fit
        pop[:] = offspring + listElitism
        fits = [ind.fitness for ind in pop]


        best_ind = tools.selBest(pop, 1)[0]
        print(best_ind)
        print(best_ind.fitness)
    end = time.time()
    print("Gen %s, Best individual is %s, %s in %s s" % (g, best_ind, best_ind.fitness,(end-start)))
    
    pool.close()
